# Get missing taxids for spore shape

Code for obtaining missing taxids (NCBI IDs) for values in the Genome_ID and Accession_ID columns of the "spore shape" dataset of bugphyzz.

In [284]:
suppressMessages({
    library(dplyr)
    library(data.table)
})

In [90]:
## This is the updated file created by Jennifer Wokaty
spore_shape_url <- "https://docs.google.com/spreadsheets/d/1tZnjvgRrv5ww88TVgjzZvcoR3SvLu9gi/export?format=xlsx"
spore_shape_file <- tempfile()
download.file(spore_shape_url, spore_shape_file)
spore_shape <- readxl::read_xlsx(spore_shape_file, na = "NA")

Get the Genome and Accession IDs with missing taxids:

In [152]:
## Genome IDs (start with GCF; they're from the assembly database)
assembly_ids_1 <- spore_shape %>% 
    filter(is.na(NCBI_ID), !is.na(Genome_ID)) %>% 
    pull(Genome_ID) %>% 
    unique()

## Accession IDs (they're from the SRA database)
sra_ids <- spore_shape %>% 
    filter(is.na(NCBI_ID), grepl("^ERS", Accession_ID)) %>% 
    pull(Accession_ID) %>% 
    unique()

## Some genome ids start with GCA insteead of GCF
assembly_ids_2 <- spore_shape %>% 
    filter(is.na(NCBI_ID), grepl("^GCA", Accession_ID))  %>% 
    pull(Accession_ID) %>% 
    unique()

In [85]:
length(assembly_ids_1)

[1] 131

In [87]:
length(sra_ids)

[1] 12

In [88]:
length(assembly_ids_2)

[1] 4

## Get taxids and taxa names

Let's define a function to help retrieving the taxids using the "efetch" command from the NCBI e-utilities. More
information at this site:https://www.ncbi.nlm.nih.gov/books/NBK179288/. This post in biostars could be useful
as well: https://www.biostars.org/p/353500/

In [219]:
get_taxids_from_assembly_or_accession <- function(x, db) {
    
    ## this function requires efetch installed in a linux system
    
    if (db == "assembly") {
        fields <- "AssemblyAccession,Taxid,Organism"
    }
    
    if (db == "sra") {
        fields <- "Sample@acc,Organism@taxid,Organism@ScientificName"
    }
    
    output <- vector("list", length(x))
    names(output) <- x
    for (i in seq_along(output)) {
        arg_string <- paste0(
            "-db ", db, " -id ", x[i], " -format docsum ",
            "| xtract -pattern DocumentSummarySet",
            " -element ", fields
        )
    
        tryCatch(
            error = function(e) NULL, {
                output[[i]] <- system2(
                    "efetch", arg_string, stdout = TRUE
                    ) |>
                {\(y) sub("\\(.+$", "", y)}()
                }
        )
    }
    purrr::discard(output, ~!length(.x)) |>
        toTable(db = db)
    
        
}

toTable <- function(x, db = "id") {
    
    if (db == "assembly") {
        id_col <- "Genome_ID"
    }
    
    if (db == "sra") {
        id_col <- "Accession_ID"
    }
    
    col_names <- c(id_col, "NCBI_ID", "Taxon_name")
    
    output <- vector("list", length(x))
    for (i in seq_along(output)) {
        output[[i]] <- x[[i]] |>
            strsplit(split = "\t") |>
            unlist() |>
            matrix(nrow = 1) |>
            dplyr::as_tibble()
    }
    dplyr::bind_rows(output) |>
        magrittr::set_colnames(col_names)
}

In [222]:
##  This can take a while because the information is fetched one by one
assembly_ids_1_taxids <- 
    get_taxids_from_assembly_or_accession(assembly_ids_1, db = "assembly")
head(assembly_ids_1_taxids)

Genome_ID,NCBI_ID,Taxon_name
<chr>,<chr>,<chr>
GCF_000332735.1,1194526,Staphylococcus warneri SG1
GCF_000798825.1,1339243,Carnobacterium sp. ZWU0011
GCF_000271405.2,768486,Enterococcus hirae ATCC 9790
GCF_000205205.1,749517,Enterococcus faecalis TX1467
GCF_000014365.1,373153,Streptococcus pneumoniae D39
GCF_000213825.1,936154,Streptococcus parauberis KCTC 11537


In [220]:
assembly_ids_2_taxids 
    <- get_taxids_from_assembly_or_accession(assembly_ids_2, db = "assembly")
assembly_ids_2_taxids

Genome_ID,NCBI_ID,Taxon_name
<chr>,<chr>,<chr>
GCA_001311805.1,1293599,Lentibacillus juripiscarius JCM 12147
GCA_001310515.1,1460650,Bacillus sp. JCM 19056
GCA_001310675.1,1481930,Oceanobacillus sp. JCM 19060
GCA_001310555.1,1460652,Bacillus sp. JCM 19058


In [221]:
sra_ids_taxids 
    <- get_taxids_from_assembly_or_accession(sra_ids, db = "sra")
head(sra_ids_taxids)

Accession_ID,NCBI_ID,Taxon_name
<chr>,<chr>,<chr>
ERS852512,162156,uncultured Bacteroides sp.
ERS852513,823,Parabacteroides distasonis
ERS852418,162156,uncultured Bacteroides sp.
ERS852560,823,Parabacteroides distasonis
ERS852380,823,Parabacteroides distasonis
ERS852397,338188,Bacteroides finegoldii


In [223]:
dim(assembly_ids_1_taxids)

[1] 126   3

In [224]:
dim(assembly_ids_2_taxids)

[1] 4 3

In [225]:
dim(sra_ids_taxids)

[1] 12  3

In [229]:
retrieved_ids_tbl <- bind_rows(assembly_ids_1_taxids, assembly_ids_2_taxids, sra_ids_taxids) |>
    relocate(NCBI_ID, Genome_ID, Accession_ID, Taxon_name)
head(retrieved_ids_tbl)

NCBI_ID,Genome_ID,Accession_ID,Taxon_name
<chr>,<chr>,<chr>,<chr>
1194526,GCF_000332735.1,NA,Staphylococcus warneri SG1
1339243,GCF_000798825.1,NA,Carnobacterium sp. ZWU0011
768486,GCF_000271405.2,NA,Enterococcus hirae ATCC 9790
749517,GCF_000205205.1,NA,Enterococcus faecalis TX1467
373153,GCF_000014365.1,NA,Streptococcus pneumoniae D39
936154,GCF_000213825.1,NA,Streptococcus parauberis KCTC 11537


In [231]:
## save the file
## it's better to back this data now since it can take a while to fetch the data
## again
readr::write_tsv(retrieved_ids_tbl, "spore_shape_retrieved_ids_tbl.tsv")

In [275]:
## Some Ids in the Accession_ID column should be in the Genome ID
## These entries start with GCA instead of GCF
spore_shape_edited <- spore_shape  %>% 
    mutate(
        Genome_ID = ifelse(grepl("^GCA", Accession_ID), Accession_ID, Genome_ID),
        Accession_ID = ifelse(grepl("^GCA", Accession_ID), NA, Accession_ID)
    )

In [370]:
x <- retrieved_ids_tbl %>% 
    filter(!is.na(Genome_ID)) %>% 
    left_join(
        spore_shape_edited, by = "Genome_ID", na_matches = "never",
        suffix = c("", ".y")
    ) %>% 
    select(-ends_with(".y")) %>% 
    mutate(NCBI_ID = as.integer(NCBI_ID))

In [371]:
y <- retrieved_ids_tbl %>% 
    mutate(NCBI_ID = as.integer(NCBI_ID)) %>% 
    filter(!is.na(Accession_ID)) %>% 
    left_join(
        spore_shape_edited, by = "Accession_ID", na_matches = "never",
        suffix = c("", ".y")
    ) %>% 
    select(-ends_with(".y"))

In [385]:
spore_shape_edited_2 <- spore_shape_edited %>% 
    mutate(NCBI_ID = as.integer(NCBI_ID)) %>% 
    filter(!Genome_ID %in% c(assembly_ids_1, assembly_ids_2)) %>% # remove old entries for Genome_ID
    filter(!Accession_ID %in% sra_ids) %>% # remove old entries for Accession_ID
    bind_rows(x)  %>% # add new entries for Genome_ID
    bind_rows(y) # add new entries for Accession_ID

Let's confirm that some of the new entires look as they should

**For Genome IDs starting with GCF**

In [398]:
## original dataset
spore_shape %>% 
    filter(Genome_ID %in% assembly_ids_1) %>% 
    select(-Attribute_source) %>% # this column occupies too much space
    head()

NCBI_ID,Genome_ID,Accession_ID,Taxon_name,Attribute,Attribute_value,Evidence,Confidence_interval
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,GCF_000350165.2,NA,Halanaerobiales,endospore,FALSE,EXP,Usually
NA,GCF_000350165.3,NA,Halanaerobiales,endospore,FALSE,EXP,Usually
NA,GCF_000350165.4,NA,Halanaerobiales,endospore,FALSE,EXP,Usually
NA,GCF_000350165.5,NA,Halanaerobiales,endospore,FALSE,EXP,Usually
NA,GCF_000350165.6,NA,Halanaerobiales,endospore,FALSE,EXP,Usually
NA,GCF_000332735.1,NA,Bacillales,endospore,FALSE,EXP,Usually


In [397]:
## edited dataset
spore_shape_edited_2  %>%
filter(Genome_ID %in% assembly_ids_1) %>% 
    select(-Attribute_source) %>% 
    head()

NCBI_ID,Genome_ID,Accession_ID,Taxon_name,Attribute,Attribute_value,Evidence,Confidence_interval
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1194526,GCF_000332735.1,NA,Staphylococcus warneri SG1,endospore,FALSE,EXP,Usually
1339243,GCF_000798825.1,NA,Carnobacterium sp. ZWU0011,endospore,FALSE,EXP,Usually
768486,GCF_000271405.2,NA,Enterococcus hirae ATCC 9790,endospore,FALSE,EXP,Usually
749517,GCF_000205205.1,NA,Enterococcus faecalis TX1467,endospore,FALSE,EXP,Usually
373153,GCF_000014365.1,NA,Streptococcus pneumoniae D39,endospore,FALSE,EXP,Usually
936154,GCF_000213825.1,NA,Streptococcus parauberis KCTC 11537,endospore,FALSE,EXP,Usually


**For Genome IDs starting with GCA**

In [408]:
## original
spore_shape %>% 
    filter(Accession_ID %in% assembly_ids_2) %>% 
    select(-Attribute_source) %>% 
    head()

NCBI_ID,Genome_ID,Accession_ID,Taxon_name,Attribute,Attribute_value,Evidence,Confidence_interval
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,GCA_001311805.1,Bacillales,endospore,TRUE,EXP,Usually
NA,NA,GCA_001310515.1,Bacillales,endospore,TRUE,EXP,Usually
NA,NA,GCA_001310675.1,Bacillales,endospore,TRUE,EXP,Usually
NA,NA,GCA_001310555.1,Bacillales,endospore,TRUE,EXP,Usually


In [409]:
## edited
spore_shape_edited_2 %>% 
    filter(Genome_ID %in% assembly_ids_2) %>% 
    select(-Attribute_source) %>% 
    head()

NCBI_ID,Genome_ID,Accession_ID,Taxon_name,Attribute,Attribute_value,Evidence,Confidence_interval
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1293599,GCA_001311805.1,NA,Lentibacillus juripiscarius JCM 12147,endospore,TRUE,EXP,Usually
1460650,GCA_001310515.1,NA,Bacillus sp. JCM 19056,endospore,TRUE,EXP,Usually
1481930,GCA_001310675.1,NA,Oceanobacillus sp. JCM 19060,endospore,TRUE,EXP,Usually
1460652,GCA_001310555.1,NA,Bacillus sp. JCM 19058,endospore,TRUE,EXP,Usually


**For Accession_ID**

In [399]:
## orginal
spore_shape %>% 
    filter(Accession_ID %in% sra_ids) %>% 
    select(-Attribute_source) %>% 
    head()

NCBI_ID,Genome_ID,Accession_ID,Taxon_name,Attribute,Attribute_value,Evidence,Confidence_interval
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,ERS852512,Bacteroidetes,endospore,FALSE,EXP,Usually
NA,NA,ERS852513,Bacteroidetes,endospore,FALSE,EXP,Usually
NA,NA,ERS852418,Bacteroidetes,endospore,FALSE,EXP,Usually
NA,NA,ERS852560,Bacteroidetes,endospore,FALSE,EXP,Usually
NA,NA,ERS852380,Bacteroidetes,endospore,FALSE,EXP,Usually
NA,NA,ERS852397,Bacteroidetes,endospore,FALSE,EXP,Usually


In [403]:
## edited
spore_shape_edited_2 %>% 
    filter(Accession_ID %in% sra_ids) %>% 
    select(-Attribute_source) %>% 
    head()

NCBI_ID,Genome_ID,Accession_ID,Taxon_name,Attribute,Attribute_value,Evidence,Confidence_interval
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
162156,NA,ERS852512,uncultured Bacteroides sp.,endospore,FALSE,EXP,Usually
823,NA,ERS852513,Parabacteroides distasonis,endospore,FALSE,EXP,Usually
162156,NA,ERS852418,uncultured Bacteroides sp.,endospore,FALSE,EXP,Usually
823,NA,ERS852560,Parabacteroides distasonis,endospore,FALSE,EXP,Usually
823,NA,ERS852380,Parabacteroides distasonis,endospore,FALSE,EXP,Usually
338188,NA,ERS852397,Bacteroides finegoldii,endospore,FALSE,EXP,Usually


The dataset is five elments shorter because some Genome IDs do not exist neither in the ncbi
or the orginial source of the attributes. Probably these entries were created by AutoFill:

In [387]:
dim(spore_shape)

[1] 1547    9

In [388]:
dim(spore_shape_edited_2)

[1] 1542    9

In [393]:
assembly_ids_1[!assembly_ids_1 %in% retrieved_ids_tbl$Genome_ID]

[1] "GCF_000350165.2" "GCF_000350165.3" "GCF_000350165.4" "GCF_000350165.5"
[5] "GCF_000350165.6"

In [410]:
## Save the new dataset as tsv
readr::write_tsv(spore_shape_edited_2, "spore_shape_edited.tsv")